# Logsitc Regression From Scratch

In this notebook, we will code the logistic regression model from scratch to understand the theory behind this commonly used machine learning model.

In the end, we will compare the result of the logistic regression model we build from scratch with the one from scikit-learn.

YT: https://youtu.be/gN79XvB7vTo

## Implementation

Input data ("y" table) has "m" data points, "n" columns (features or independent variables), and "n" + 1 total of betas.

In [15]:
# import library
import random
import numpy as np

For the main function, we perform the steps as follows:
- Transform "X" and "y" dataframe into numpy arrays.
- Initialize the parameters using `initialize_params` based on the dimension of the input data ("n").
- Use the next helper function, `compute_gradients`, to derive gradients at each beta.
- Use the `update_params` function to update the beta values using the gradients.
- Repeat the step for the number of iterations that we've specified.

In [16]:
# main function
def logistic_regression(X, y, iterations = 100, learning_rate = 0.01):
    X = X.to_numpy()
    y = y.to_numpy()
    
    n, m = len(X[0]), len(X)
    beta_0, beta_other = initialize_params(n)
    
    for _ in range(iterations):
        gradient_beta_0, gradient_beta_other = compute_gradients(X, y, beta_0, beta_other, n, m)
        beta_0, beta_other = update_params(beta_0, beta_other, gradient_beta_0, gradient_beta_other, learning_rate)
    return beta_0, beta_other

For the `initialize_params` function, we initialize "beta_0" as 0, and "beta_other" is a vector with the size of "n" that holds all the other randomly initialized betas.

In [17]:
# helper function: initialize params
def initialize_params(n):
    beta_0 = 0
    beta_other = [random.random() for _ in range(n)]
    return beta_0, beta_other

`compute_gradients` is the core of the algorithm where we compute gradients for all betas:
- Initialized all gradient betas as 0.
- We loop through all data points and add a gradient contributed by each data point to those variables. Inside the `for` loop:
    - First, we obtain the prediction "y_i_hat" using the `logistic_function`.
    - Get the difference (residual) between the prediction ("y_i_hat") and the observation ("y[i]").
    - The gradient for "beta_0" is just the residual, but the gradient for "beta_other" is the residual multiplied by the feature.
    - Accumulate the gradient from all data points (using "+=").
    - Lastly, divide each data point's gradient by "m" so the gradient computed at the end will be the average over all data points.

In [18]:
# helper function: compute gradient
def compute_gradients(X, y, beta_0, beta_other, n , m):
    gradient_beta_0 = 0
    gradient_beta_other = [0] * n
    
    for i, point in enumerate(X):
        y_i_hat = logistic_function(point, beta_0, beta_other)
        residual =  y_i_hat - y[i]
        for j, feature in enumerate(point):
            gradient_beta_other[j] += residual * feature / m
        gradient_beta_0 += residual / m
            
    return gradient_beta_0, gradient_beta_other

We apply the logistic function (Sigmoid function) in `logistic_function` to calculate the prediction.

In [19]:
# helper function: logistic function
def logistic_function(point, beta_0, beta_other):
    return 1 / (1 + np.exp(-(beta_0 + point.dot(beta_other))))

We use `update_params` to update all the betas using the gradient we obtained. We don't add gradients to betas, but we scale the gradient by multiplying it with the learning rate (a rate of speed where the gradient moves during a gradient descent; a learning rate too high will make gradient descent unstable, too low will make it slow to converge).

In [20]:
# helper function: update parameters
def update_params(beta_0, beta_other, gradient_beta_0, gradient_beta_other, learning_rate):
    beta_0 -= gradient_beta_0 * learning_rate
    for i in range(len(beta_other)):
        beta_other[i] -= (gradient_beta_other[i] * learning_rate)
    return beta_0, beta_other

## Model Comparison

For this section, we will be using the popular Iris dataset to perform the comparison between our logistic regression model and the scikit-learn `LogisticRegression` model. Because our model can only be used for binary classification, we will only predict only 1 species (Iris Setosa) instead of all 3 species that are available in the dataset. The metric we will use for our comparison will be the outputted betas from both models.

### Library & Data Preparation

In [21]:
# load library
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [22]:
# load dataset
df = pd.read_csv('data/Iris_Dataset.csv')
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [23]:
# transform dataset for binary classification
df['Species'].replace(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], [1, 0, 0], inplace = True)

In [24]:
# normalize data
df_normalized = (df - df.min()) / (df.max() - df.min()) # min-max normalization
df_normalized.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,0.222222,0.625000,0.067797,0.041667,1.0
1,0.166667,0.416667,0.067797,0.041667,1.0
2,0.111111,0.500000,0.050847,0.041667,1.0
3,0.083333,0.458333,0.084746,0.041667,1.0
4,0.194444,0.666667,0.067797,0.041667,1.0


In [25]:
# separate independent and dependent variables
X = df_normalized[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]] # independent variables
y = df_normalized["Species"] # dependent variable

### Models Building

In [26]:
# sklearn logistic regression model
model_sklearn  = LogisticRegression().fit(X, y)

print("Intercept | Constant of Logistic Regression Equation: ", model_sklearn.intercept_)
print("Coefficient of Logistic Regression Equation: ", model_sklearn.coef_)

Intercept | Constant of Logistic Regression Equation:  [1.48912751]
Coefficient of Logistic Regression Equation:  [[-1.61033977  1.88360913 -3.31081248 -3.24444535]]


In [27]:
# our logistic regression model
model_scratch = logistic_regression(X, y, iterations = 1000, learning_rate = 0.1)

print("Intercept | Constant of Logistic Regression Equation: ", model_scratch[0])
print("Coefficient of Logistic Regression Equation: ", model_scratch[1])

Intercept | Constant of Logistic Regression Equation:  1.260213131716376
Coefficient of Logistic Regression Equation:  [-1.6857031217591012, 2.726790512780263, -3.5746320923493586, -3.687798116572437]


As we can see from the results above, both the intercepts (beta_0) and the coefficients (beta_other) are very similar between the two models. Hence, we have successfully built a logistic regression from scratch!

One thing to note is our model is very sensitive to any characteristic the dataset might have. For example, for us to get the betas from our scratch model to be as similar to the scikit-learn's model, we had to perform (min-max) normalization on the dataset.